<a href="https://colab.research.google.com/github/Vidcito/Examen-SegundoBimestre/blob/main/SistemaDeEcuaciones_DavidPuga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import logging
from sys import stdout
from datetime import datetime
import numpy as np
import os

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)

logging.info(f"{datetime.now()}")

In [50]:
# ####################################################################
def gauss_jordan(Ab: np.ndarray) -> np.ndarray:
    """Resuelve un sistema de ecuaciones lineales mediante el método de Gauss-Jordan.

    ## Parameters

    ``Ab``: matriz aumentada del sistema de ecuaciones lineales. Debe ser de tamaño n-by-(n+1), donde n es el número de incógnitas.

    ## Return

    ``solucion``: vector con la solución del sistema de ecuaciones lineales.

    """
    if not isinstance(Ab, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        Ab = np.array(Ab, dtype=float)
    assert Ab.shape[0] == Ab.shape[1] - 1, "La matriz A debe ser de tamaño n-by-(n+1)."
    n = Ab.shape[0]

    for i in range(0, n):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if Ab[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(Ab[pi, i]) < abs(Ab[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = Ab[i, :].copy()
            Ab[i, :] = Ab[p, :].copy()
            Ab[p, :] = _aux

        # --- Eliminación: loop por fila
        for j in range(n):
            if i == j:
                continue
            m = Ab[j, i] / Ab[i, i]
            Ab[j, i:] = Ab[j, i:] - m * Ab[i, i:]

        logging.info(f"\n{Ab}")

    if Ab[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

    # --- Sustitución hacia atrás
    solucion = np.zeros(n)

    for i in range(n - 1, -1, -1):
        solucion[i] = Ab[i, -1] / Ab[i, i]

    return solucion

In [57]:
%autoreload 2
from src import matriz_aumentada

A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
b = [1, -2, 3, 4]

Ab = matriz_aumentada(A, b)
gauss_jordan(Ab)

array([ 3., -6.,  6., -2.])

In [71]:
# prompt: # prompt: def inv_matrix(A: np.ndarray) -> np.ndarray:
# #     """Inversión de una matriz cuadrada mediante método de Gauss-Jordan.
# #     ## Parameters
# #     ``A``: matriz cuadrada de tamaño n x n.
# #     ## Return
# #     ``A_inv``: matriz inversa de A.
# #     """
# #     # COMPLETAR
# #     # Debe basarse en la función gauss_jordan

def inv_matrix(A: np.ndarray) -> np.ndarray:
    """Inversión de una matriz cuadrada mediante método de Gauss-Jordan.
    ## Parameters
    ``A``: matriz cuadrada de tamaño n x n.
    ## Return
    ``A_inv``: matriz inversa de A.
    """
    if not isinstance(A, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        A = np.array(A, dtype=float)
    assert A.shape[0] == A.shape[1], "La matriz A debe ser cuadrada."
    n = A.shape[0]
    A_inv = np.eye(n)
    Ab = np.concatenate((A, A_inv), axis=1)
    for i in range(0, n):
        p = None
        for pi in range(i, n):
            if Ab[pi, i] == 0:
                continue
            if p is None:
                p = pi
                continue
            if abs(Ab[pi, i]) < abs(Ab[p, i]):
                p = pi
        if p is None:
          raise ValueError("No existe solución única.")

        if p != i:
            _aux = Ab[i, :].copy()
            Ab[i, :] = Ab[p, :].copy()
            Ab[p, :] = _aux
        for j in range(n):
          if i == j:
            continue
          m = Ab[j,i]/Ab[i,i]
          Ab[j,:] = Ab[j,:] - m*Ab[i,:]
        Ab[i,:] = Ab[i,:] / Ab[i,i]

    return Ab[:, n:]


In [72]:
A = [[2, -3], [-1, 1]]
inv_matrix(A)

array([[-1., -3.],
       [-1., -2.]])

In [69]:
A = [
    [4, 0, 0, 5],
    [1, 0, 4, 0],
    [3, 4, 1, 3],
    [1, 3, 3, 0],
]
inv_matrix(A)

array([[-36.,  45.,  60., -80.],
       [  3.,  -4.,  -5.,   7.],
       [  9., -11., -15.,  20.],
       [ 29., -36., -48.,  64.]])

In [67]:
A = [
    [0, 0, 0, 0, 0, 0, 1, -1],
    [0, 1, -1, 1, 0, -1, 0, 1],
    [-1, -1, 0, 0, 2, 1, 0, 0],
    [-1, -1, -1, 1, 2, 0, 0, 1],
    [-1, 1, 1, 0, -1, -1, 0, 2],
    [0, 1, 0, 0, -1, -1, 0, 0],
    [1, -1, -1, 1, 2, 1, 0, 2],
    [2, 0, 0, 0, 0, 1, 2, 0],
]
inv_matrix(A)

array([[ 2., -1.,  0., -1.,  0.,  2.,  2., -1.],
       [ 0.,  1.,  1., -1.,  0.,  0.,  0.,  0.],
       [ 6., -1.,  0., -3.,  1.,  1.,  4., -3.],
       [ 6.,  1., -1., -3.,  1., -3.,  3., -3.],
       [ 2., -1.,  1., -1.,  0.,  3.,  2., -1.],
       [-2.,  2.,  0.,  0.,  0., -4., -2.,  1.],
       [-1.,  0.,  0.,  1.,  0.,  0., -1.,  1.],
       [-2.,  0.,  0.,  1.,  0.,  0., -1.,  1.]])

In [68]:
A = [
    [1, 0, 0, 0, -1, 0, 0, -1, 1, -1],
    [1, 1, 0, -1, -1, 1, 0, 0, 1, -1],
    [-1, 0, -1, 0, 0, 0, -1, 1, 0, 0],
    [0, 0, -1, 0, -1, -1, 1, 0, 1, 0],
    [-1, 0, 0, -1, 1, 1, 1, 1, 0, -1],
    [1, 0, 0, 1, -1, -1, -1, 1, -1, 0],
    [1, 1, 1, 0, 1, 0, -1, -1, -1, 1],
    [1, 1, 1, 1, 0, 0, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 0, -1, -1, 0, 0],
    [0, 0, -1, -1, -1, 0, 1, 1, 1, -1],
]
inv_matrix(A)

array([[ 14.,  -8.,   9.,  -4.,   0.,  -4.,   9.,   7.,  -8.,   3.],
       [ -2.,   2.,  -1.,   2.,   1.,   1.,  -1.,  -1.,   1.,  -2.],
       [-27.,  14., -18.,   5.,  -2.,   7., -17., -13.,  16.,  -2.],
       [ 12.,  -6.,   8.,  -2.,   1.,  -3.,   7.,   6.,  -7.,   0.],
       [  6.,  -4.,   4.,  -2.,   0.,  -2.,   4.,   3.,  -3.,   2.],
       [ 18.,  -9.,  12.,  -4.,   1.,  -5.,  11.,   9., -11.,   1.],
       [  8.,  -4.,   5.,  -1.,   1.,  -2.,   5.,   4.,  -5.,   0.],
       [ -5.,   2.,  -3.,   0.,  -1.,   1.,  -3.,  -2.,   3.,   1.],
       [-11.,   5.,  -7.,   1.,  -2.,   2.,  -7.,  -5.,   7.,   1.],
       [  1.,  -1.,   1.,  -1.,  -1.,  -1.,   1.,   1.,  -1.,   1.]])